In [1]:
org_name = 'mbry'
org_full_name = 'Methylocystis_bryophila'

In [2]:
import myModules.buildmodel as BM

## Add others

### energy

In [3]:
import cobra
import pandas as pd
new_model = cobra.io.read_sbml_model(org_name+'\\07_'+org_name+'_biomass_model.xml')

In [4]:
new_model

<Model Methylocystis_bryophila at 0x186d9285760>

In [5]:
new_model.reactions.get_by_id('Htcp').remove_from_model()

BM.add_rxn_by_string(new_model,'NADH16','NADH dehydrogenase (ubiquinone-8 & 3 protons)','4.0 h_c + nadh_c + q8_c --> nad_c + q8h2_c + 3.0 h_p')
BM.add_annotation_from_BIGGurl(new_model.reactions.get_by_id('NADH16'))
BM.add_rxn_by_string(new_model,'CYOR_q8','Ubiquinol-cytochrome c oxidoreductase Complex III','2.0 ficytC_p + 2.0 h_c + q8h2_c --> 4.0 h_p + 2.0 focytC_p + q8_c')
new_model.reactions.get_by_id('CYOR_q8').annotation = BM.biggunimodel.reactions.get_by_id('CYOR_q8_m').annotation
BM.add_rxn_by_string(new_model,'CYOO3','Cytochrome c oxidase He 2','6.0 h_c + 0.5 o2_c + 2.0 focytC_p --> h2o_c + 4.0 h_p + 2.0 ficytC_p')
BM.add_annotation_from_BIGGurl(new_model.reactions.get_by_id('CYOO3'))
BM.add_rxn_by_string(new_model,'ATPS','ATPase cytosolic','adp_c + pi_c + h_e --> atp_c + h2o_c')
BM.add_annotation_from_BIGGurl(new_model.reactions.get_by_id('ATPS'))
BM.add_rxn_by_string(new_model,'TRDR','Thioredoxin reductase (NADPH)','h_c + nadph_c + trdox_c --> nadp_c + trdrd_c')
BM.add_annotation_from_BIGGurl(new_model.reactions.get_by_id('TRDR'))
BM.add_rxn_by_string(new_model,'TRDRf','Thioredoxin:ferredoxin reductase','2.0 h_c + trdox_c + 2.0 fdxrd_c <--> trdrd_c + 2.0 fdxox_c')
BM.add_annotation_from_BIGGurl(new_model.reactions.get_by_id('TRDRf'))

### Ethylmalonyl pathway

In [6]:
BM.add_rxn_by_string(new_model,'EthyMalon','Ethylmalonyl pathway gap','aacoa_c + co2_c + h_c --> glx_c + ppcoa_c')

### others

In [7]:
new_model.reactions.get_by_id('IMPC').bounds = (-1000,1000)
new_model.reactions.get_by_id('ILETA').bounds = (-1000,1000)
new_model.reactions.get_by_id('LEUTA').bounds = (-1000,1000)
new_model.reactions.get_by_id('POR_syn').bounds = (-1000,1000)
 


In [8]:
new_model.summary()

In [9]:
cobra.io.write_sbml_model(new_model, org_name+'\\08_'+org_name+'_energy_model.xml')
cobra.io.save_json_model(new_model, org_name+'\\08_'+org_name+'_energy_model.json')

## gapfill

In [10]:
new_model = cobra.io.read_sbml_model(org_name+'\\08_'+org_name+'_energy_model.xml')

In [11]:
new_model

<Model Methylocystis_bryophila at 0x186d76ebbb0>

In [12]:
mko_model = cobra.io.read_sbml_model('mko\\10_mko_gapfill_model_V2.xml')

In [13]:
mko_rxn = [rr.id for rr in mko_model.reactions]
mbry_rxn = [rr.id for rr in new_model.reactions]
not_in_mbry = [rr for rr in mko_rxn if rr not in mbry_rxn]
print(len(not_in_mbry))
in_mbry = [rr for rr in mko_rxn if rr in mbry_rxn]
print(len(in_mbry))

115
670


In [14]:
for rr in in_mbry:
    (a,b) = mko_model.reactions.get_by_id(rr).bounds
    (c,d) = new_model.reactions.get_by_id(rr).bounds
    if a != c or b != d:
        print(rr,a,b,c,d)

GHMT1 0.0 1000.0 -1000.0 1000.0
CS -1000.0 1000.0 0.0 1000.0


In [15]:
check_model = new_model.copy()

In [16]:
for rr in not_in_mbry:
    if rr in ['DM_mko_biomass_c','RU5PS']:
        continue
    check_model.add_reactions([mko_model.reactions.get_by_id(rr)])

In [17]:
check_model.summary()

In [18]:
check_model.reactions.NIT1b.bounds = (0,0)

In [19]:
check_model.summary()

In [20]:
a = check_model.optimize().fluxes

In [21]:
import_rxn = []
for rr in not_in_mbry:
    if rr in ['DM_mko_biomass_c','RU5PS']:
        continue
    if not a[rr]==0:
        import_rxn.append(rr)
        BM.print_rxn(check_model,rr)
        print('\t'+str(a[rr]))

BFBP	Fructose-bisphosphatase	fdp_B_c + h2o_c --> f6p_B_c + pi_c
	0.30460430110196535
PFKppi	Phosphofructokinase (ppi-dependent)	f6p_B_c + ppi_c <=> fdp_B_c + h_c + pi_c
	-0.02553422230460478
ICDHx	Isocitrate dehydrogenase (NAD)	icit_c + nad_c <=> akg_c + co2_c + nadh_c
	0.5885328406014402
ADNK1	Adenosine kinase	adn_c + atp_c --> adp_c + amp_c + h_c
	0.013621871088887141
DHORDfum	Fumarate dependent DHORD	dhor__S_c + fum_c <=> orot_c + succ_c
	2.235813052520879
DHORD_NAD	Dihydoorotic acid dehydrogenase	dhor__S_c + nad_c <=> nadh_c + orot_c
	-2.1552294561525542
CPPPGO	Coproporphyrinogen oxidase (O2 required)	cpppg3_c + 2.0 h_c + o2_c --> 2.0 co2_c + 2.0 h2o_c + pppg9_c
	8.796834699526752e-05
THRD_L	L-threonine deaminase	thr__L_c --> 2obut_c + nh4_c
	0.15399200935722893
SERD_L	L-serine deaminase	ser__L_c --> nh4_c + pyr_c
	5.731192511776793
THRA	Threonine aldolase	thr__L_c <=> acald_c + gly_c
	7.55885265282197
CHORM	Chorismate mutase	chor_c <=> pphn_c
	0.2021076514488548
ME1	Malic enzyme (

In [22]:
for rxn in import_rxn:
    rxn = check_model.reactions.get_by_id(rxn)
    if rxn.annotation.get('kegg.reaction'):
        print(rxn.id+'\t'+str(rxn.annotation.get('kegg.reaction')))
    else:
        print(rxn.id)

BFBP	R04780
PFKppi	R00764
ICDHx	R00709
ADNK1	R00185
DHORDfum	R01867
DHORD_NAD	R01869
CPPPGO	R03220
THRD_L	R00996
SERD_L	['R00220', 'R00223']
THRA	R00751
CHORM	R01715
ME1	R00214
PEPCK	R00431
G5SADs	R03314
USHD	R04549
KDOPS	R03254
A5PISO	R01530
KDOPP	R03350
OMCDC	R01652
HP	R03013
HISTDb	R01168
ALATA_D	R01148
UAAGDS	R02788
UGMDDS	R04617
FMNRx	R05705
PMDPHT	R07280
Htcp
IPDPS
HDS
BPNT	R00188
CAT	R00009


In [23]:
for rr in not_in_mbry:
    if rr in ['DM_mko_biomass_c','RU5PS']:
        continue
    if rr in import_rxn:
        continue
    else:
        check_model.remove_reactions([rr])

In [24]:
check_model.summary()

In [25]:
could_be_removed = []
for rr in import_rxn:
    check_model.remove_reactions([rr])
    if check_model.optimize().objective_value==0:
        check_model.add_reactions([mko_model.reactions.get_by_id(rr)])
    else:
        print(rr)
        could_be_removed.append(rr)
        #check_model.add_reactions([mko_model.reactions.get_by_id(rr)])

BFBP
ICDHx
ADNK1
DHORDfum
THRD_L
SERD_L
ME1
PEPCK
FMNRx
Htcp
CAT


In [26]:
cobra.io.write_sbml_model(check_model, org_name+'\\'+org_name+'_check_model.xml')

In [27]:
gapfill_rxn = []
for rr in import_rxn:
    if rr not in could_be_removed:
        gapfill_rxn.append(rr)
for rr in gapfill_rxn:
    BM.print_rxn(check_model,rr)

PFKppi	Phosphofructokinase (ppi-dependent)	f6p_B_c + ppi_c <=> fdp_B_c + h_c + pi_c
DHORD_NAD	Dihydoorotic acid dehydrogenase	dhor__S_c + nad_c <=> nadh_c + orot_c
CPPPGO	Coproporphyrinogen oxidase (O2 required)	cpppg3_c + 2.0 h_c + o2_c --> 2.0 co2_c + 2.0 h2o_c + pppg9_c
THRA	Threonine aldolase	thr__L_c <=> acald_c + gly_c
CHORM	Chorismate mutase	chor_c <=> pphn_c
G5SADs	L-glutamate 5-semialdehyde dehydratase (spontaneous)	glu5sa_c <=> 1pyr5c_c + h2o_c + h_c
USHD	UDP-sugar hydrolase	h2o_c + u23ga_c --> 2.0 h_c + lipidX_c + ump_c
KDOPS	3-deoxy -D-manno-octulosonic -acid 8-phosphate synthase	ara5p_c + h2o_c + pep_c --> kdo8p_c + pi_c
A5PISO	Arabinose-5-phosphate isomerase	ru5p__D_c <=> ara5p_c
KDOPP	3-deoxy-manno-octulosonate-8-phosphatase	h2o_c + kdo8p_c --> kdo_c + pi_c
OMCDC	2-Oxo-4-methyl-3-carboxypentanoate decarboxylation	3c4mop_c + h_c --> 4mop_c + co2_c
HP	Histidinol-phosphatase, cytosol	h2o_c + h_c + hisp_c --> histd_c + pi_c
HISTDb	Histidinol dehydrogenase (histidine forming)

In [28]:
check_model.optimize().fluxes.to_csv(org_name+'\\'+org_name+'_check_model_flux.csv')

In [29]:
check_model.reactions.THRA.remove_from_model()

In [30]:
bigg_model = cobra.io.load_json_model('.\\BIGG\\universal_model_modified_onlyC.json')

In [31]:
solution = cobra.flux_analysis.gapfill(check_model, bigg_model, demand_reactions=False,iterations=5)

In [32]:
def print_sol(solution,bigg_model=bigg_model):
    i=0
    for sol in solution:
        print('-------soultion_%s--------'%(i))
        for rxn in sol:
            BM.print_rxn(bigg_model,rxn.id)
        i+=1

In [33]:
print(gapfill_rxn)

['PFKppi', 'DHORD_NAD', 'CPPPGO', 'THRA', 'CHORM', 'G5SADs', 'USHD', 'KDOPS', 'A5PISO', 'KDOPP', 'OMCDC', 'HP', 'HISTDb', 'ALATA_D', 'UAAGDS', 'UGMDDS', 'PMDPHT', 'IPDPS', 'HDS', 'BPNT']


In [34]:
print_sol(solution)

-------soultion_0--------
RBCh	Ribulose-bisphosphate carboxylase	o2_c + rb15bp_c --> 2pglyc_c + 3pg_c + 2.0 h_c
-------soultion_1--------
THRA	Threonine aldolase	thr__L_c --> acald_c + gly_c
-------soultion_2--------
GLYCL_2	Glycine cleavage system, cytosol	co2_c + mlthf_c + nadh_c + nh4_c --> gly_c + nad_c + thf_c
-------soultion_3--------
SK_coa_c	Sink coa[c]	coa_c --> 
-------soultion_4--------
GLYFEM		glyclt_c + h_c --> ac_c + co2_c + h2o_c + succ_c


In [35]:
# real_gapfill_rxn = ['PFKppi', 'DHORD_NAD', 'CPPPGO', 'CHORM', 'G5SADs', 'USHD', 'KDOPS', 'A5PISO', 'KDOPP', 'OMCDC', 'HP', 'HISTDb', 'ALATA_D', 'UAAGDS', 'UGMDDS', 'PMDPHT', 'IPDPS', 'HDS', 'BPNT']

In [36]:
for rr in gapfill_rxn:
    if rr == 'THRA':
        continue
    new_model.add_reactions([mko_model.reactions.get_by_id(rr)])
new_model.reactions.get_by_id('GLYCL').bounds = (-1000,1000)
new_model.reactions.get_by_id('NIT1b').bounds = (0,0)
new_model.reactions.get_by_id('APPAT').bounds = (0,0)       
new_model.reactions.get_by_id('ACS').bounds = (-1000,1000)  
new_model.reactions.get_by_id('ACKr').bounds = (-1000,0)    
new_model.reactions.get_by_id('PPDK').bounds = (0,1000)     
new_model.reactions.get_by_id('EX_no2_e').bounds = (0,0)      

In [37]:
new_model.summary()

In [38]:
cobra.io.write_sbml_model(new_model, org_name+'\\09_'+org_name+'_complate_model_V1.xml')
cobra.io.save_json_model(new_model, org_name+'\\09_'+org_name+'_complate_model_V1.json')

## clean up

run in cmd

```
cd mbry
memote run 09_mbry_complate_model_V1.xml
```

In [39]:
new_model = cobra.io.read_sbml_model(org_name+'\\09_'+org_name+'_complate_model_V1.xml')

In [40]:
for mete in new_model.metabolites:
    if len(mete.reactions) == 0:
        print(mete.id)
        new_model.remove_metabolites(mete)

In [41]:
duplicate_meta =   [["uacgala_c","udpacgal_c"]]
for metas in duplicate_meta:
    for rxn in new_model.metabolites.get_by_id(metas[1]).reactions:
        rxn.add_metabolites({new_model.metabolites.get_by_id(metas[0]):rxn.metabolites.get(new_model.metabolites.get_by_id(metas[1]))})
    new_model.remove_metabolites(new_model.metabolites.get_by_id(metas[1]))

In [42]:
new_model.summary()

In [43]:
new_model

<Model Methylocystis_bryophila at 0x186826a9940>

In [44]:
cobra.io.write_sbml_model(new_model, org_name+'\\10_'+org_name+'_clean_model.xml')
cobra.io.save_json_model(new_model, org_name+'\\10_'+org_name+'_clean_model.json')

In [45]:
new_model.optimize().fluxes.to_csv(org_name+'\\10_'+org_name+'_clean_model_flux.csv')

## add gene rule

In [46]:
new_model = cobra.io.read_sbml_model(org_name+'\\10_'+org_name+'_clean_model.xml')
gene_r = pd.read_csv('genome\\Methylocystis_bryophila_genome\\Gene_R.txt',index_col=None,sep='\t')

In [47]:
def add_to_dir(my_dir, key, value):
    if key in my_dir:
        if isinstance(my_dir[key], list):
            my_dir[key].append(value)
        else:
            my_dir[key] = [my_dir[key], value]
    else:
        my_dir[key] = value

In [48]:
R_gene_dir = {}
for i in range(len(gene_r)):
    gene = gene_r.iloc[i,0]
    Rid = gene_r.iloc[i,2]
    add_to_dir(R_gene_dir, Rid, gene)


In [49]:
for rxn in new_model.reactions:
    keggid = rxn.annotation.get('kegg.reaction')
    if type(keggid)==list:
        keggid = keggid[0]
    if keggid in R_gene_dir.keys():
        if type(R_gene_dir[keggid])==list:
            rxn.gene_reaction_rule = ' or '.join(R_gene_dir[keggid])
        else:
            rxn.gene_reaction_rule = R_gene_dir[keggid]

In [50]:
new_model

<Model Methylocystis_bryophila at 0x18682f5b340>

In [51]:
cobra.io.write_sbml_model(new_model, org_name+'\\11_'+org_name+'_complate_model_V2.xml')
cobra.io.save_json_model(new_model, org_name+'\\11_'+org_name+'_complate_model_V2.json')